In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Chaitanya\\Documents\\ML\\Indian-Medical-Leaf-Clf\\Medicinal-Leaf-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Chaitanya\\Documents\\ML\\Indian-Medical-Leaf-Clf\\Medicinal-Leaf-Classification'

In [5]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")      #Replace with saved model 

OSError: No file or directory found at artifacts/training/model.h5

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    params_seed: int

In [10]:
from MedicineLeafClassifier.constants import *
from MedicineLeafClassifier.utils.common import read_yaml,create_directories,save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH  ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_validation_config(self) ->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/indian-medicinal-leaf-image-dataset/Medicinal Leaf dataset",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_seed=self.params.SEED
        )

        return eval_config

In [15]:
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def get_valid_data(self):
        self.valid_data = tf.keras.preprocessing.image_dataset_from_directory(
             self.config.training_data,
             labels = 'inferred',
             label_mode = 'int',
             color_mode = 'rgb',
            class_names = None,
             batch_size = self.config.params_batch_size,
             image_size= (self.config.params_image_size[0], self.config.params_image_size[1]),
             shuffle=True,
             seed=self.config.params_seed,
             validation_split=0.1,
             subset='validation',
             )
        AUTOTUNE = tf.data.AUTOTUNE

        # train_data = train_data.cache().prefetch(buffer_size = AUTOTUNE)

        self.valid_data = self.valid_data.cache().prefetch(buffer_size = AUTOTUNE)

        # return valid_data
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.get_valid_data()
        self.score = model.evaluate(self.valid_data)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e